In [106]:
! pip install bs4

In [107]:
from flask import Flask, request, jsonify, render_template
import random
from tensorflow.keras.models import load_model
import numpy as np
import librosa
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from bs4 import BeautifulSoup as SOUP
import re
import requests as HTTP

In [108]:
app = Flask(__name__, template_folder='template', static_folder='static')
model=load_model(r'C:\Users\This PC\Downloads\prototype\model.h5')

In [109]:
@app.route('/')
def home():
    return render_template('prototype.html')

In [110]:
@app.route('/predict_',methods=['POST'])
def predict_():

    # get file from POST request and save it
    audio_file = request.files["file"]
    file_name = str(random.randint(0, 100000))
    audio_file.save(file_name)

    Features = pd.read_csv(r'C:\Users\This PC\Downloads\prototype\features.csv')
    Y = Features['labels'].values
    encoder = OneHotEncoder()
    Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
    featurelive = mfccs
    livedf2 = featurelive
    livedf2= pd.DataFrame(data=livedf2)
    livedf2 = livedf2.stack().to_frame().T
    twodim= np.expand_dims(livedf2, axis=2)



    prediction = model.predict(twodim)
    livepredictions = (encoder.inverse_transform((prediction)))


    liveprediction=livepredictions.tolist()
    string_version = " ".join(str(x) for x in liveprediction)
    print(string_version)
    emotion=string_version.split("'")
    emotion=emotion[1].split("_")
    gender=emotion[0]
    emotion=emotion[1]


    male_genre=["https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=history,war&colors=color",
           "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=adventure,mystery&colors=color",
           "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=drama&colors=color",
           "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=biography,documentary&colors=color",
           "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=action,comedy,crime&colors=color",
           "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=action,thriller&colors=color",
           "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=family&colors=color",
           "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=horror&colors=color"]

    female_genre=["https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=adventure&colors=color",
             "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=adventure,comedy,romance&colors=color",
             "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=drama,fantasy&colors=color",
             "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=music,musical&colors=color",
             "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=action,comedy,crime&colors=color",
             "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=action,thriller&colors=color",
             "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=family,game-show&colors=color",
             "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,tv_episode,tv_special,tv_miniseries,documentary,short,tv_short&release_date=2016-01-01,&user_rating=7.5,10.0&genres=horror&colors=color"]
    

    if(gender == "male"):    
         # IMDb Url for movie against emotion Neutral
         if(emotion == "neutral"):
             urlhere = male_genre[0]
  
         # IMDb Url for movie against emotion Happy
         elif(emotion == "happy"):
             urlhere = male_genre[1]
  
         # IMDb Url for movie against emotion Sad
         elif(emotion == "sad"):
             urlhere = male_genre[2]
  
         # IMDb Url for movie against emotion Calm
         elif(emotion == "calm"):
             urlhere = male_genre[3]
  
         # IMDb Url for movie against emotion Surprised
         elif(emotion == "surprised"):
             urlhere = male_genre[4]

         # IMDb Url for movie against emotion Angry
         elif(emotion == "angry"):
             urlhere = male_genre[5]

         # IMDb Url for  movie against emotion Fearful
         elif(emotion == "fear"):
             urlhere = male_genre[6]

         # IMDb Url for movie against emotion Disgust
         elif(emotion == "disgust"):
             urlhere = male_genre[7]
        
    elif(gender=="female"):
         if(emotion == "neutral"):
             urlhere = female_genre[0]
  
         # IMDb Url for movie against emotion Happy
         elif(emotion == "happy"):
             urlhere = female_genre[1]
  
         # IMDb Url for movie against emotion Sad
         elif(emotion == "sad"):
             urlhere = female_genre[2]
  
         # IMDb Url for movie against emotion Calm
         elif(emotion == "calm"):
             urlhere = female_genre[3]
  
         # IMDb Url for movie against emotion Surprised
         elif(emotion == "surprised"):
             urlhere = female_genre[4]

         # IMDb Url for movie against emotion Angry
         elif(emotion == "angry"):
             urlhere = female_genre[5]

         # IMDb Url for movie against emotion Fearful
         elif(emotion == "fear"):
             urlhere = female_genre[6]

         # IMDb Url for movie against emotion Disgust
         elif(emotion == "disgust"):
             urlhere = female_genre[7]
        

    response = HTTP.get(urlhere)
    data = response.text
    soup = SOUP(data, "lxml")


    vbLf='\n'
    TITLES = []
    titles = soup.find_all('h3')
    for t in titles:
      TITLES.append(t.text[4:].replace(vbLf,""))   

    URL_list = []
    for item in soup.find_all(attrs={'class':'lister-item-header'}):
          for link in item.find_all('a',href=True):
             href=link.get('href')
             URL_list.append("https://www.imdb.com/"+href+"?ref_=adv_li_tt")  



    RATINGS = []
    ratings =soup.find_all("div",{'class':"inline-block ratings-imdb-rating"})
    for r in ratings:
          RATINGS.append(r.text.replace(vbLf,""))

    # created the dataframe to store the scrapped information
    data = pd.DataFrame(zip(TITLES, RATINGS, URL_list), columns = ["Title", "Ratings", "URL"])              
    
    return render_template('prototype.html',column_names=data.columns.values, row_data=list(data.values.tolist()), zip=zip)

In [111]:
@app.route('/results',methods=['POST'])
def results():
    data = request.get_json(force=True)
    prediction = model.predict(data.values())
    
    return jsonify(prediction)

In [112]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Oct/2021 18:20:08] "GET / HTTP/1.1" 200 -


['female_sad']


127.0.0.1 - - [06/Oct/2021 18:20:47] "POST /predict_ HTTP/1.1" 200 -
